## **AllenNLP demo**

This is a demo for prediction venue based on title and abstract of the paper

reference: https://github.com/allenai/allennlp-as-a-library-example

In [1]:
import json
from typing import Iterator, List, Dict, Optional
import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# for dataset reader
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField, LabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer
from allennlp.data.vocabulary import Vocabulary

# read pretrained embedding from AWS S3
from allennlp.modules.token_embedders.embedding import _read_embeddings_from_text_file

# for building model
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules import FeedForward
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.nn import InitializerApplicator, RegularizerApplicator
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

## **Create classes for the model**

Generally, we need to implement 2 classes for AllenNLP including

- `DatasetReader`: to read dataset and return `Instance` class
- `Model`: input `Instance` class and return output prediction

`Model` consists of the Sequence to Vector model (`Seq2Vec`)

<img src="figures/bilstm.png" width="300"/>


and we use the combination of vectors to predict venue

<img src="figures/venue_prediction.png" width="300"/>

In [2]:
class PublicationDatasetReader(DatasetReader):
    """
    DatasetReader for publication and venue dataaet
    """
    def __init__(self, 
                 tokenizer: Tokenizer = None,
                 token_indexers: Dict[str, TokenIndexer] = None, 
                 lazy: bool = False) -> None:
        super().__init__(lazy)
        self._tokenizer = tokenizer or WordTokenizer()
        self._token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}

    def _read(self, file_path: str) -> Iterator[Instance]:
        """
        Read publication and venue dataset in JSON format
        
        Data is in the following format:
            {"title": ..., "paperAbstract": ..., "venue": ...}
        """
        with open(cached_path(file_path), "r") as data_file:
            for line in data_file:
                line = line.strip("\n")
                if not line:
                    continue
                paper_json = json.loads(line)
                title = paper_json['title']
                abstract = paper_json['paperAbstract']
                venue = paper_json['venue']
                yield self.text_to_instance(title, abstract, venue)
        
    def text_to_instance(self, 
                         title: str, 
                         abstract: str, 
                         venue: str=None) -> Instance:
        """
        Turn title, abstract, and venue to instance
        """
        tokenized_title = self._tokenizer.tokenize(title)
        tokenized_abstract = self._tokenizer.tokenize(abstract)
        title_field = TextField(tokenized_title, self._token_indexers)
        abstract_field = TextField(tokenized_abstract, self._token_indexers)
        fields = {'title': title_field, 
                  'abstract': abstract_field}
        if venue is not None:
            fields['label'] = LabelField(venue)
        return Instance(fields)

In [3]:
class AcademicPaperClassifier(Model):
    """
    Model to classify venue based on input title and abstract
    """
    def __init__(self, 
                 vocab: Vocabulary,
                 text_field_embedder: TextFieldEmbedder,
                 title_encoder: Seq2VecEncoder,
                 abstract_encoder: Seq2VecEncoder,
                 classifier_feedforward: FeedForward,
                 initializer: InitializerApplicator = InitializerApplicator(),
                 regularizer: Optional[RegularizerApplicator] = None) -> None:
        super(AcademicPaperClassifier, self).__init__(vocab, regularizer)
        self.text_field_embedder = text_field_embedder
        self.num_classes = self.vocab.get_vocab_size("labels")
        self.title_encoder = title_encoder
        self.abstract_encoder = abstract_encoder
        self.classifier_feedforward = classifier_feedforward
        self.metrics = {
                "accuracy": CategoricalAccuracy(),
                "accuracy3": CategoricalAccuracy(top_k=3)
        }
        self.loss = torch.nn.CrossEntropyLoss()
        initializer(self)
    
    def forward(self, 
                title: Dict[str, torch.LongTensor],
                abstract: Dict[str, torch.LongTensor],
                label: torch.LongTensor = None) -> Dict[str, torch.Tensor]:
        
        embedded_title = self.text_field_embedder(title)
        title_mask = get_text_field_mask(title)
        encoded_title = self.title_encoder(embedded_title, title_mask)

        embedded_abstract = self.text_field_embedder(abstract)
        abstract_mask = get_text_field_mask(abstract)
        encoded_abstract = self.abstract_encoder(embedded_abstract, abstract_mask)

        logits = self.classifier_feedforward(torch.cat([encoded_title, encoded_abstract], dim=-1))
        class_probabilities = F.softmax(logits, dim=-1)
        argmax_indices = np.argmax(class_probabilities.cpu().data.numpy(), axis=-1)
        labels = [self.vocab.get_token_from_index(x, namespace="labels") for x in argmax_indices]
        output_dict = {
            'logits': logits, 
            'class_probabilities': class_probabilities,
            'predicted_label': labels
        }
        if label is not None:
            loss = self.loss(logits, label)
            for metric in self.metrics.values():
                metric(logits, label)
            output_dict["loss"] = loss

        return output_dict

## **Read dataset**

- `cached_path`: can cache the file locally
- `BasicTextFieldEmbedder` takes a mapping from index names to embeddings

In [4]:
train_data_path = "https://s3-us-west-2.amazonaws.com/allennlp/datasets/academic-papers-example/train.jsonl"
validation_data_path = "https://s3-us-west-2.amazonaws.com/allennlp/datasets/academic-papers-example/dev.jsonl"
pretrained_file = "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.100d.txt.gz"

In [5]:
reader = PublicationDatasetReader()

In [6]:
instance = reader.text_to_instance("This is a great paper.", 
                                   "Indeed, this is a great paper of all time", 
                                   "Nature")

In [7]:
train_dataset = reader.read(cached_path(train_data_path))
validation_dataset = reader.read(cached_path(validation_data_path))

15000it [00:35, 428.37it/s]
2000it [00:04, 419.87it/s]


In [8]:
# building vocabulary
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

100%|██████████| 17000/17000 [00:02<00:00, 6340.02it/s]


In [9]:
# load pre-trained embedding
embedding_matrix = _read_embeddings_from_text_file(file_uri=pretrained_file, 
                                                   embedding_dim=100, 
                                                   vocab=vocab)

400000it [00:04, 80881.50it/s] 


In [10]:
print(embedding_matrix.size()) 

torch.Size([64247, 100])


In [11]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
num_classes = len(vocab.get_index_to_token_vocabulary('labels'))

In [12]:
# embedding
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'), 
                            embedding_dim=EMBEDDING_DIM,
                            weight=embedding_matrix)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [13]:
lstm_title = PytorchSeq2VecWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, 
                                                 batch_first=True, bidirectional=True))
lstm_abstract = PytorchSeq2VecWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, 
                                                    batch_first=True, bidirectional=True))
feed_forward = torch.nn.Linear(2 * 2 * HIDDEN_DIM, num_classes)

In [14]:
model = AcademicPaperClassifier(vocab,
                                word_embeddings, 
                                lstm_title, 
                                lstm_abstract, 
                                feed_forward)

In [15]:
optimizer = optim.SGD(model.parameters(), lr=0.005)

In [16]:
iterator = BucketIterator(batch_size=64, 
                          sorting_keys=[("abstract", "num_tokens"), 
                                        ("title", "num_tokens")])
iterator.index_with(vocab) # index with the created vocabulary

In [17]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=validation_dataset,
    patience=2,
    num_epochs=5,
    serialization_dir='output'
)

In [18]:
trainer.train()

loss: 1.0962 ||: 100%|██████████| 235/235 [40:56<00:00, 10.45s/it]  
loss: 1.0874 ||: 100%|██████████| 32/32 [00:07<00:00,  4.16it/s]
loss: 1.0858 ||: 100%|██████████| 235/235 [39:35<00:00, 10.11s/it]  
loss: 1.0808 ||: 100%|██████████| 32/32 [00:05<00:00,  5.42it/s]
loss: 1.0815 ||: 100%|██████████| 235/235 [39:27<00:00, 10.07s/it] 
loss: 1.0766 ||: 100%|██████████| 32/32 [00:05<00:00,  5.62it/s]
loss: 1.0779 ||: 100%|██████████| 235/235 [45:42<00:00, 11.67s/it]  
loss: 1.0725 ||: 100%|██████████| 32/32 [00:05<00:00,  5.63it/s]
loss: 1.0742 ||: 100%|██████████| 235/235 [40:22<00:00, 10.31s/it]  
loss: 1.0688 ||: 100%|██████████| 32/32 [00:05<00:00,  5.44it/s]


{'best_epoch': 4,
 'peak_cpu_memory_MB': 1412.108288,
 'training_duration': '3:26:36.009762',
 'training_start_epoch': 0,
 'training_epochs': 4,
 'epoch': 4,
 'training_loss': 1.0742415935435194,
 'training_cpu_memory_MB': 1412.108288,
 'validation_loss': 1.0688360258936882,
 'best_validation_loss': 1.0688360258936882}

## **Prediction**

Lastly, we can also write the prediction class `PaperClassifierPredictor` which take input any `json_dict` and return the `Instance`. The AllenNLP will take care of the prediction.

In [ ]:
from allennlp.common.util import JsonDict
from allennlp.predictors.predictor import Predictor

In [20]:
class PaperClassifierPredictor(Predictor):
    """"
    Predictor wrapper for the AcademicPaperClassifier
    """
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        title = json_dict['title']
        abstract = json_dict['paperAbstract']
        instance = self._dataset_reader.text_to_instance(title=title, abstract=abstract)
        return instance

In [21]:
predictor = PaperClassifierPredictor(model, dataset_reader=reader)

In [22]:
prediction_output = predictor.predict_json(
    {
        "title": "Know What You Don't Know: Unanswerable Questions for SQuAD", 
        "paperAbstract": "Extractive reading comprehension systems can often locate the correct answer to a question in a context document, but they also tend to make unreliable guesses on questions for which the correct answer is not stated in the context. Existing datasets either focus exclusively on answerable questions, or use automatically generated unanswerable questions that are easy to identify. To address these weaknesses, we present SQuAD 2.0, the latest version of the Stanford Question Answering Dataset (SQuAD). SQuAD 2.0 combines existing SQuAD data with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD 2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering. SQuAD 2.0 is a challenging natural language understanding task for existing models: a strong neural system that gets 86% F1 on SQuAD 1.1 achieves only 66% F1 on SQuAD 2.0."
    }
)

print(prediction_output)

{'logits': [0.07398353517055511, 0.08393894135951996, -0.1481025218963623], 'class_probabilities': [0.35576409101486206, 0.35932356119155884, 0.2849124073982239], 'predicted_label': 'AI'}


## **Load model for prediction**

Here, we trained the model and save it in `output` folder using 

```
allennlp train example_training.json -s output --include-package venue
```

`venue` is a library that we created where we make AllenNLP as a library. We can load trained model (`model.tar.gz`) from `serialization_dir` (`output`) and use it to predict the classes.

In [23]:
from allennlp.models.archival import load_archive
from allennlp.predictors.predictor import Predictor
from venue.venue_predictor import PaperClassifierPredictor
from venue.venue_reader import PublicationDatasetReader
from venue.venue_classifier import AcademicPaperClassifier

archive = load_archive('output/model.tar.gz')
venue_predictor = Predictor.from_archive(archive, 'venue_predictor')

//anaconda3/envs/Project_SC/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [24]:
prediction_output = venue_predictor.predict_json(
    {
        "title": "Know What You Don't Know: Unanswerable Questions for SQuAD", 
        "paperAbstract": "Extractive reading comprehension systems can often locate the correct answer to a question in a context document, but they also tend to make unreliable guesses on questions for which the correct answer is not stated in the context. Existing datasets either focus exclusively on answerable questions, or use automatically generated unanswerable questions that are easy to identify. To address these weaknesses, we present SQuAD 2.0, the latest version of the Stanford Question Answering Dataset (SQuAD). SQuAD 2.0 combines existing SQuAD data with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD 2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering. SQuAD 2.0 is a challenging natural language understanding task for existing models: a strong neural system that gets 86% F1 on SQuAD 1.1 achieves only 66% F1 on SQuAD 2.0."
    }
)

print(prediction_output)

{'logits': [1.3005162477493286, 0.6157347559928894, -2.2390246391296387], 'class_probabilities': [0.6522191166877747, 0.3288491368293762, 0.018931739032268524], 'predicted_label': 'ACL'}


In [25]:
venue_predictor._model.vocab.get_index_to_token_vocabulary('labels') # all classes

{0: 'ACL', 1: 'AI', 2: 'ML'}

## **Predict which journals to submit from Medline**

We  do a fun experiment where we train the same model to classify 
publications from sample 110 journals from MEDLINE. We got accuracy of 64.9 percent on the validation dataset.

In [26]:
import torch
from allennlp.models.archival import load_archive
from allennlp.predictors.predictor import Predictor
from venue.venue_predictor import PaperClassifierPredictor
from venue.venue_reader import PublicationDatasetReader
from venue.venue_classifier import AcademicPaperClassifier
from allennlp.common.file_utils import cached_path
import torch.nn.functional as F

In [29]:
archive = load_archive(cached_path('https://s3-us-west-2.amazonaws.com/allennlp-tutorial/model.tar.gz'))
venue_predictor = Predictor.from_archive(archive, 'venue_predictor')



  0%|          | 0/791998690 [00:00<?, ?B/s]

  0%|          | 17408/791998690 [00:00<1:56:53, 112916.56B/s]

  0%|          | 52224/791998690 [00:00<1:39:15, 132976.78B/s]

  0%|          | 121856/791998690 [00:00<1:17:56, 169326.10B/s]

  0%|          | 261120/791998690 [00:00<58:53, 224088.11B/s]  

  0%|          | 303104/791998690 [00:00<57:32, 229316.14B/s]

  0%|          | 339968/791998690 [00:00<55:40, 237014.11B/s]

  0%|          | 373760/791998690 [00:01<56:38, 232931.73B/s]

  0%|          | 404480/791998690 [00:01<1:19:32, 165867.62B/s]

  0%|          | 429056/791998690 [00:01<1:20:28, 163927.00B/s]

  0%|          | 470016/791998690 [00:01<1:11:16, 185086.40B/s]

  0%|          | 504832/791998690 [00:01<1:24:05, 156882.56B/s]

  0%|          | 609280/791998690 [00:02<1:04:40, 203919.23B/s]

  0%|          | 661504/791998690 [00:02<1:08:52, 191485.02B/s]

  0%|          | 713728/791998690 [00:02<59:04, 223257.70B/s]  

  0%|          | 783360/791998690 [00:02<50:29, 26

  0%|          | 3603456/791998690 [00:22<1:11:37, 183461.28B/s]

  0%|          | 3638272/791998690 [00:22<1:07:40, 194166.93B/s]

  0%|          | 3673088/791998690 [00:22<1:06:06, 198726.44B/s]

  0%|          | 3707904/791998690 [00:23<1:04:49, 202692.86B/s]

  0%|          | 3742720/791998690 [00:23<1:02:43, 209446.86B/s]

  0%|          | 3776512/791998690 [00:23<59:09, 222093.78B/s]  

  0%|          | 3800064/791998690 [00:23<1:07:23, 194908.14B/s]

  0%|          | 3829760/791998690 [00:23<1:08:17, 192357.60B/s]

  0%|          | 3864576/791998690 [00:23<1:05:20, 201030.36B/s]

  0%|          | 3899392/791998690 [00:23<1:03:14, 207683.90B/s]

  0%|          | 3920896/791998690 [00:24<1:24:14, 155916.90B/s]

  1%|          | 3969024/791998690 [00:24<1:11:27, 183804.59B/s]

  1%|          | 3991552/791998690 [00:24<1:10:49, 185421.71B/s]

  1%|          | 4013056/791998690 [00:24<1:17:14, 170037.62B/s]

  1%|          | 4038656/791998690 [00:24<1:18:02, 168260.86B/s]

  1%|     

  1%|          | 7224320/791998690 [00:40<49:46, 262761.58B/s]

  1%|          | 7259136/791998690 [00:40<50:13, 260447.24B/s]

  1%|          | 7311360/791998690 [00:40<47:44, 273977.67B/s]

  1%|          | 7363584/791998690 [00:40<45:22, 288227.97B/s]

  1%|          | 7398400/791998690 [00:41<47:00, 278219.48B/s]

  1%|          | 7450624/791998690 [00:41<45:31, 287215.11B/s]

  1%|          | 7485440/791998690 [00:41<1:04:38, 202250.12B/s]

  1%|          | 7536640/791998690 [00:41<56:48, 230164.49B/s]  

  1%|          | 7572480/791998690 [00:41<56:38, 230799.40B/s]

  1%|          | 7607296/791998690 [00:42<56:54, 229712.30B/s]

  1%|          | 7642112/791998690 [00:42<57:21, 227927.33B/s]

  1%|          | 7694336/791998690 [00:42<52:09, 250600.08B/s]

  1%|          | 7729152/791998690 [00:42<53:42, 243347.21B/s]

  1%|          | 7763968/791998690 [00:42<54:46, 238637.59B/s]

  1%|          | 7816192/791998690 [00:42<50:40, 257949.34B/s]

  1%|          | 7851008/791998690 [

  1%|▏         | 10775552/791998690 [01:03<1:09:54, 186237.38B/s]

  1%|▏         | 10795008/791998690 [01:03<1:48:50, 119625.85B/s]

  1%|▏         | 10845184/791998690 [01:04<1:28:57, 146359.77B/s]

  1%|▏         | 10865664/791998690 [01:04<1:31:26, 142379.00B/s]

  1%|▏         | 10884096/791998690 [01:04<1:29:48, 144954.56B/s]

  1%|▏         | 10914816/791998690 [01:04<1:26:18, 150841.90B/s]

  1%|▏         | 10932224/791998690 [01:04<1:26:37, 150285.11B/s]

  1%|▏         | 10967040/791998690 [01:04<1:21:42, 159328.88B/s]

  1%|▏         | 11001856/791998690 [01:05<1:14:54, 173778.06B/s]

  1%|▏         | 11021312/791998690 [01:05<1:15:57, 171351.96B/s]

  1%|▏         | 11054080/791998690 [01:05<1:12:18, 180019.07B/s]

  1%|▏         | 11088896/791998690 [01:05<1:08:06, 191114.56B/s]

  1%|▏         | 11123712/791998690 [01:05<1:08:24, 190243.84B/s]

  1%|▏         | 11158528/791998690 [01:05<1:05:13, 199536.47B/s]

  1%|▏         | 11179008/791998690 [01:05<1:06:37, 195311.89B

  2%|▏         | 14744576/791998690 [01:21<1:04:18, 201455.45B/s]

  2%|▏         | 14779392/791998690 [01:22<1:04:03, 202209.93B/s]

  2%|▏         | 14800896/791998690 [01:22<1:09:05, 187470.14B/s]

  2%|▏         | 14831616/791998690 [01:22<1:08:02, 190377.57B/s]

  2%|▏         | 14866432/791998690 [01:22<1:06:03, 196066.55B/s]

  2%|▏         | 14901248/791998690 [01:22<1:03:50, 202889.24B/s]

  2%|▏         | 14936064/791998690 [01:22<1:02:02, 208755.85B/s]

  2%|▏         | 14970880/791998690 [01:22<1:00:18, 214724.81B/s]

  2%|▏         | 15005696/791998690 [01:23<1:15:27, 171629.54B/s]

  2%|▏         | 15056896/791998690 [01:23<1:03:01, 205458.67B/s]

  2%|▏         | 15081472/791998690 [01:23<1:11:33, 180942.23B/s]

  2%|▏         | 15102976/791998690 [01:23<1:12:37, 178283.19B/s]

  2%|▏         | 15127552/791998690 [01:23<1:16:18, 169678.11B/s]

  2%|▏         | 15162368/791998690 [01:23<1:10:45, 182995.70B/s]

  2%|▏         | 15197184/791998690 [01:24<1:08:35, 188756.08B

...

 96%|█████████▌| 756447232/791998690 [44:58<04:27, 133119.86B/s]

 96%|█████████▌| 756482048/791998690 [44:58<03:54, 151194.74B/s]

 96%|█████████▌| 756499456/791998690 [44:58<04:15, 138695.95B/s]

 96%|█████████▌| 756534272/791998690 [44:58<03:47, 156226.63B/s]

 96%|█████████▌| 756568064/791998690 [44:58<03:23, 173693.18B/s]

 96%|█████████▌| 756587520/791998690 [44:58<03:46, 156057.96B/s]

 96%|█████████▌| 756621312/791998690 [44:59<03:26, 171266.93B/s]

 96%|█████████▌| 756656128/791998690 [44:59<03:10, 185543.54B/s]

 96%|█████████▌| 756708352/791998690 [44:59<02:45, 213542.82B/s]

 96%|█████████▌| 756743168/791998690 [44:59<02:40, 219563.64B/s]

 96%|█████████▌| 756795392/791998690 [44:59<02:24, 244382.23B/s]

 96%|█████████▌| 756847616/791998690 [44:59<02:11, 267498.97B/s]

 96%|█████████▌| 756899840/791998690 [45:00<02:01, 289947.23B/s]

 96%|█████████▌| 756969472/791998690 [45:00<01:48, 323876.23B/s]

 96%|█████████▌| 757039104/791998690 [45:00<01:38, 354503.73B/s]

 96%|█████

 97%|█████████▋| 765294592/791998690 [45:20<10:30, 42360.91B/s]

 97%|█████████▋| 765307904/791998690 [45:20<11:54, 37330.44B/s]

 97%|█████████▋| 765325312/791998690 [45:21<09:32, 46575.02B/s]

 97%|█████████▋| 765342720/791998690 [45:21<07:50, 56671.61B/s]

 97%|█████████▋| 765360128/791998690 [45:21<06:39, 66755.50B/s]

 97%|█████████▋| 765377536/791998690 [45:21<05:49, 76128.68B/s]

 97%|█████████▋| 765394944/791998690 [45:21<04:52, 91024.82B/s]

 97%|█████████▋| 765429760/791998690 [45:21<04:11, 105568.66B/s]

 97%|█████████▋| 765447168/791998690 [45:21<04:06, 107897.25B/s]

 97%|█████████▋| 765464576/791998690 [45:22<03:59, 110803.28B/s]

 97%|█████████▋| 765499392/791998690 [45:22<03:23, 129920.93B/s]

 97%|█████████▋| 765516800/791998690 [45:22<03:30, 125978.27B/s]

 97%|█████████▋| 765551616/791998690 [45:22<03:03, 144390.94B/s]

 97%|█████████▋| 765586432/791998690 [45:22<02:43, 161505.99B/s]

 97%|█████████▋| 765621248/791998690 [45:22<02:29, 176756.28B/s]

 97%|█████████▋| 

 98%|█████████▊| 773837824/791998690 [45:37<00:34, 527983.93B/s]

 98%|█████████▊| 773897216/791998690 [45:37<00:36, 493077.54B/s]

 98%|█████████▊| 773951488/791998690 [45:37<00:39, 455600.39B/s]

 98%|█████████▊| 774011904/791998690 [45:38<00:42, 426811.99B/s]

 98%|█████████▊| 774081536/791998690 [45:38<00:41, 433151.74B/s]

 98%|█████████▊| 774151168/791998690 [45:38<00:40, 439786.88B/s]

 98%|█████████▊| 774220800/791998690 [45:38<00:39, 444649.99B/s]

 98%|█████████▊| 774290432/791998690 [45:38<00:39, 447364.77B/s]

 98%|█████████▊| 774360064/791998690 [45:38<00:39, 450935.42B/s]

 98%|█████████▊| 774429696/791998690 [45:39<00:38, 458631.81B/s]

 98%|█████████▊| 774499328/791998690 [45:39<00:38, 456463.81B/s]

 98%|█████████▊| 774568960/791998690 [45:39<00:37, 458851.01B/s]

 98%|█████████▊| 774638592/791998690 [45:39<00:37, 460101.65B/s]

 98%|█████████▊| 774708224/791998690 [45:39<00:33, 509200.32B/s]

 98%|█████████▊| 774761472/791998690 [45:39<00:36, 473209.67B/s]

 98%|█████

 99%|█████████▊| 780400640/791998690 [45:55<00:14, 798665.46B/s]

 99%|█████████▊| 780574720/791998690 [45:55<00:13, 868194.04B/s]

 99%|█████████▊| 780731392/791998690 [45:55<00:12, 917522.13B/s]

 99%|█████████▊| 780922880/791998690 [45:55<00:11, 986231.34B/s]

 99%|█████████▊| 781114368/791998690 [45:56<00:10, 1057604.89B/s]

 99%|█████████▊| 781323264/791998690 [45:56<00:09, 1120207.27B/s]

 99%|█████████▊| 781532160/791998690 [45:56<00:08, 1195505.19B/s]

 99%|█████████▊| 781758464/791998690 [45:56<00:07, 1389665.81B/s]

 99%|█████████▊| 781913088/791998690 [45:58<00:36, 275807.78B/s] 

 99%|█████████▊| 782023680/791998690 [45:58<00:37, 264141.02B/s]

 99%|█████████▉| 782108672/791998690 [46:03<03:14, 50845.30B/s] 

 99%|█████████▉| 782169088/791998690 [46:04<03:29, 46851.15B/s]

 99%|█████████▉| 782212096/791998690 [46:05<03:18, 49362.96B/s]

 99%|█████████▉| 782244864/791998690 [46:05<02:32, 63792.77B/s]

 99%|█████████▉| 782273536/791998690 [46:06<02:32, 63849.04B/s]

 99%|████

100%|█████████▉| 788965376/791998690 [46:23<00:43, 69349.70B/s]

100%|█████████▉| 788982784/791998690 [46:23<00:38, 78323.09B/s]

100%|█████████▉| 789000192/791998690 [46:23<00:34, 86391.76B/s]

100%|█████████▉| 789017600/791998690 [46:23<00:32, 93054.06B/s]

100%|█████████▉| 789035008/791998690 [46:24<00:30, 98283.28B/s]

100%|█████████▉| 789052416/791998690 [46:24<00:28, 102508.90B/s]

100%|█████████▉| 789069824/791998690 [46:24<00:27, 105804.04B/s]

100%|█████████▉| 789087232/791998690 [46:24<00:26, 108533.60B/s]

100%|█████████▉| 789104640/791998690 [46:24<00:26, 110158.77B/s]

100%|█████████▉| 789122048/791998690 [46:24<00:25, 111217.75B/s]

100%|█████████▉| 789139456/791998690 [46:25<00:25, 112150.38B/s]

100%|█████████▉| 789156864/791998690 [46:25<00:25, 113472.15B/s]

100%|█████████▉| 789174272/791998690 [46:25<00:22, 124191.17B/s]

100%|█████████▉| 789209088/791998690 [46:25<00:20, 135695.17B/s]

100%|█████████▉| 789226496/791998690 [46:25<00:21, 129737.66B/s]

100%|█████████▉

In [30]:
title = """
Modeling peripheral visual acuity enables discovery of gaze strategies 
at multiple time scales during natural scene search
"""
abstract = """
Like humans, monkeys make saccades nearly three times a second. 
To understand the factors guiding this frequent decision, computational models of vision 
attempt to predict fixation locations using bottom-up visual features and top-down goals. 
How do the relative influences of these factors evolve over multiple time scales? 
Here we analyzed visual features at fixations using a retinal transform that provides realistic 
visual acuity by suitably degrading visual information in the periphery. 
In a task in which monkeys searched for a Gabor target in natural scenes, we characterized 
the relative importance of bottom-up and task-relevant influences by decoding fixated from 
nonfixated image patches based on visual features. At fast time scales, we found that search 
strategies can vary over the course of a single trial, with locations of higher saliency, target-similarity, 
edge–energy, and orientedness looked at later on in the trial. At slow time scales, we found that 
search strategies can be refined over several weeks of practice, and the influence of target orientation 
was significant only in the latter of two search tasks. Critically, these results were not observed without 
applying the retinal transform. Our results suggest that saccade-guidance strategies become apparent only 
when models take into account degraded visual representation in the periphery.'
"""
prediction_output = venue_predictor.predict_json(
    {
        "title": title, 
        "paperAbstract": abstract
    }
)

In [31]:
venues = venue_predictor._model.vocab.get_index_to_token_vocabulary('labels') # all classes
venues = [venues[i] for i in range(len(venues))]

In [32]:
# rank top 5 which journal to submit
sorted(list(zip(venues, prediction_output['class_probabilities'])), 
       key=lambda x: x[1], reverse=True)[0:5]

[('Journal of vision', 0.49882569909095764),
 ('Frontiers in psychology', 0.4009263813495636),
 ('Journal of neurophysiology', 0.04838859662413597),
 ('The Journal of neuroscience : the official journal of the Society for Neuroscience',
  0.03905170410871506),
 ('PLoS computational biology', 0.006291384808719158)]

In [36]:
# Start to test
from allennlp.predictors.predictor import Predictor
archive_oei = load_archive(cached_path('https://s3-us-west-2.amazonaws.com/allennlp/models/openie-model.2018-08-20.tar.gz'))
predictor_oei = Predictor.from_archive(archive_oei)

predictor_oei.predict(
  sentence="John decided to run for office next month."
)

{'verbs': [{'verb': 'decided',
   'description': '[ARG0: John] [V: decided] [ARG1: to run for office next month] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'run',
   'description': '[ARG0: John] [BV: decided to] [V: run] [ARG1: for office] [ARG2: next month] .',
   'tags': ['B-ARG0',
    'B-BV',
    'I-BV',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARG2',
    'I-ARG2',
    'O']}],
 'words': ['John',
  'decided',
  'to',
  'run',
  'for',
  'office',
  'next',
  'month',
  '.']}